In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-face-private/image_dataset.csv
/kaggle/input/my-face-private/images/not_human/03b64f157684e1ad4ca40589e8e4c45f_large.jpg
/kaggle/input/my-face-private/images/not_human/3449809.jpg
/kaggle/input/my-face-private/images/human/Subject(3).jpg
/kaggle/input/my-face-private/images/human/Subject(1).jpg
/kaggle/input/my-face-private/images/human/Subject(2).jpg
/kaggle/input/my-face-private/images/human/Subject(8).jpg
/kaggle/input/my-face-private/images/human/Subject(5).jpg
/kaggle/input/my-face-private/images/human/Subject(4).jpg
/kaggle/input/my-face-private/images/human/Subject.jpg
/kaggle/input/my-face-private/images/human/Subject(6).jpg
/kaggle/input/my-face-private/images/human/Subject(7).jpg


**Upgrade Env**

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
strategy = tf.distribute.TPUStrategy(tpu)
batch_size = 10 * strategy.num_replicas_in_sync

In [2]:
!sudo apt update
!sudo apt upgrade -y

Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]       
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease                       0m
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]      
Get:5 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [2673 B]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:7 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [489 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]      
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.0 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3389 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2597 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2740 kB]
Ge

**Import Vision Library**

In [3]:
import tensorflow as tf
import matplotlib as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

**Check Version**

In [4]:
print(tf.__version__)
print(plt.__version__)
print(np.__version__)

2.12.0
3.7.2
1.23.5


In [5]:
from keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet', include_top=True)
model.summary()

553467096/553467096 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

# Training Step

In [6]:
from keras.layers import Flatten, Dense, Input
from keras.models import Model
from sklearn.model_selection import train_test_split

Load the VGG16 model with pre-trained weights on ImageNet dataset

In [7]:
# Remove the last layer of the model
x = model.output
x = Flatten()(x)

In [8]:
# Define x here before using it in the Dense() layer
x = Dense(256,activation='relu')(x)

Add my own custom last layer with the number of neurons equal to the number of classes you want to classify

In [9]:
predictions = Dense(2, activation='softmax')(x)

Freeze all layers except for the last few layers that you want to retrain

In [11]:
for layer in model.layers:
    layer.trainable = False

Get you own dataset

In [18]:
data_csv = pd.read_csv("/kaggle/input/my-face-private/image_dataset.csv")
trn_df, val_df = train_test_split(
    data_csv[data_csv["class"].isin(["human", "not_human"])],
    test_size=0.2,
    random_state=2,
)
val_df

,filepath,class
4,/kaggle/input/d/hakony31298/my-face-private/im...,human
1,/kaggle/input/d/hakony31298/my-face-private/im...,human
9,/kaggle/input/d/hakony31298/my-face-private/im...,not_human


In [19]:
from keras.preprocessing.image import ImageDataGenerator

# Preprocess the data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=trn_df,
        #directory='/kaggle/input/my-face-private/images',
        x_col='filepath',
        y_col='class',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
)

validation_generator = validation_datagen.flow_from_dataframe(
        dataframe=val_df,
        #directory='/kaggle/input/my-face-private/images',
        x_col='filepath',
        y_col='class',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary'
)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 8 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 3 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  warnings.warn(


In [20]:
def dataloader(
    image_paths, 
    prompts, 
    shuffle=True, 
    repeat=True, 
    batch_size=1,
):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, prompts))
    dataset = dataset.map(
        lambda x, y: (read_image(x), y),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    dataset = dataset.batch(
        batch_size, drop_remainder=True
    )

    dataset = dataset.map(
        lambda x, y: (x, (batch_tokenize(y))),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    if shuffle:
        dataset = dataset.shuffle(
            batch_size * 8, reshuffle_each_iteration = False
        )
    
    if repeat:
        dataset = dataset.repeat()
        
    return dataset.prefetch(tf.data.AUTOTUNE)

In [24]:
train_ds = dataloader(
    trn_df["filepath"].values, 
    trn_df['class'].values, 
    shuffle=True, 
    repeat=False,
    batch_size=batch_size
)

valid_ds = dataloader(
    val_df["filepath"].values, 
    val_df['class'].values, 
    shuffle=False, 
    repeat=False, 
    batch_size=batch_size
)

NameError: name 'batch_size' is not defined

Create, compile, train and save a new model with your custom last layer

In [22]:
model = Model(
    inputs=model.input, 
    outputs=predictions

)
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)
model.fit(
    train_ds 
    epochs=10, 
    batch_size=32, 
    validation_data=valid_ds
)
model.save('my_model.h5')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3574543465.py, line 12)

# Inference Step

In [ ]:
from keras.models import load_model

# Evaluation